In [1]:
from datetime import datetime

import jupyter_black
import pandas as pd

jupyter_black.load()

In [2]:
# URL of dataset CSV. Be sure to regenerate for the
# current year before running.
DATASET_INFO_CSV = (
    "https://raw.githubusercontent.com/OCHA-DAP/"
    + "hdx-analysis-scripts/gh-pages/datasets_info/datasets.csv"
)
# We only want to consider datasets 5 years or older
# e.g. In January 2022, we archived datasets created
# before 31 December 2016. UPPER_BOUND_YEAR should
# be used with an exclusive < (i.e. dataset years < UPPER_BOUND_YEAR)
UPPER_BOUND_YEAR = datetime.today().year - 5
# Max number of downloads in the past 5 years
MAX_DOWNLOADS = 1_000

In [3]:
UPPER_BOUND_YEAR

2019

In [4]:
# Takes awhile to read in because it's a large file
df = pd.read_csv(DATASET_INFO_CSV)

In [5]:
# The number of rows
df.shape[0]

26971

In [6]:
# Let's look at the column names
df.columns

Index(['name', 'title', 'id', 'downloads last 5 years', 'date created',
       'date metadata updated', 'date data updated', 'updated last 3 months',
       'updated previous quarter', 'reference period start',
       'reference period end', 'update frequency', 'fresh', 'organisation',
       'data link', 'url', 'is cod', 'tags', 'public', 'requestable',
       'archived', 'updated by cod script', 'updated by non-cod script',
       'date updated by script', 'updated_by_script<<last_modified',
       'last_modified<<updated_by_script'],
      dtype='object')

In [7]:
# We want datasets that are public and not archived
df_publicnoarchive = df.loc[(df["public"] == "Y") & (df["archived"] == "N")]
df_publicnoarchive.shape[0]

20924

In [8]:
# We only want to consider datasets that were created more than 5 years ago
df_publicnoarchive_5yo = df_publicnoarchive.loc[
    pd.to_datetime(df_publicnoarchive["date created"]).dt.year
    < UPPER_BOUND_YEAR
]
df_publicnoarchive_5yo.shape[0]

2674

In [9]:
# Confirm maximum date is < UPPER_BOUND_YEAR
df_publicnoarchive_5yo["date created"].max()

'2018-12-28T08:16:39.642635'

In [10]:
# Datasets must have < 1000 download counts
df_publicnoarchive_5yo_lt1000dl = df_publicnoarchive_5yo.loc[
    df_publicnoarchive_5yo["downloads last 5 years"] < MAX_DOWNLOADS
]
df_publicnoarchive_5yo_lt1000dl.shape[0]

1964

In [11]:
# Datasets must not be CODs
df_publicnoarchive_5yo_lt1000dl_notcod = df_publicnoarchive_5yo_lt1000dl.loc[
    df_publicnoarchive_5yo_lt1000dl["is cod"] == "N"
]
df_publicnoarchive_5yo_lt1000dl_notcod.shape[0]

1795

In [12]:
# Write to CSV
df_publicnoarchive_5yo_lt1000dl_notcod.to_csv("output.csv", index=False)